In [14]:
import requests
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from CertificationAuthority import CertificationAuthority
from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives import hashes
import util


In [15]:
ca_ports={}
ca_ports['ca2_1']=8004
ca_ports['ca1_1']=8003
ca_ports['ca2']=8002
ca_ports['ca1']=8001
ca_ports['ca_root']=8000

ca_datas={}
def get_all_ca_pub_keys():
    for name,port in ca_ports.items():
        response= requests.get(f"http://127.0.0.1:{port}/get_CA_public_key")
        pub_key=serialization.load_pem_public_key(response.content)
        ca_datas[name]={}
        ca_datas[name]["public_key"]=pub_key
        response= requests.get(f"http://127.0.0.1:{port}/get_CA_cert")
        cert=x509.load_pem_x509_certificate(response.content)
        ca_datas[name]['cert']=cert
get_all_ca_pub_keys()
for k,v in ca_datas.items():
    print(k+": "+str(v))

    


ca2_1: {'public_key': <cryptography.hazmat.backends.openssl.rsa._RSAPublicKey object at 0x7fba241546d0>, 'cert': <Certificate(subject=<Name(2.5.4.6=SG, 2.5.4.8=Singapore, 2.5.4.7=Singapore, 2.5.4.10=ca2_1, 2.5.4.3=ca2_1, )>, ...)>}
ca1_1: {'public_key': <cryptography.hazmat.backends.openssl.rsa._RSAPublicKey object at 0x7fba27c4f220>, 'cert': <Certificate(subject=<Name(2.5.4.6=SG, 2.5.4.8=Singapore, 2.5.4.7=Singapore, 2.5.4.10=ca1_1, 2.5.4.3=ca1_1, )>, ...)>}
ca2: {'public_key': <cryptography.hazmat.backends.openssl.rsa._RSAPublicKey object at 0x7fba24157dc0>, 'cert': <Certificate(subject=<Name(2.5.4.6=SG, 2.5.4.8=Singapore, 2.5.4.7=Singapore, 2.5.4.10=ca2, 2.5.4.3=ca2, )>, ...)>}
ca1: {'public_key': <cryptography.hazmat.backends.openssl.rsa._RSAPublicKey object at 0x7fba27c4e080>, 'cert': <Certificate(subject=<Name(2.5.4.6=SG, 2.5.4.8=Singapore, 2.5.4.7=Singapore, 2.5.4.10=ca1, 2.5.4.3=ca1, )>, ...)>}
ca_root: {'public_key': <cryptography.hazmat.backends.openssl.rsa._RSAPublicKey obje

In [16]:
#ca_root use port 8000
#ca1 use port 8001
#ca2 use port 8002
#ca1_1 use port 8003
#ca1_2 use port 8004

# cleint 1 cert sign by root
# 1. generate RSA key
rsa_key=util.generate_ras_key()

# 2. create certificate sign request and sign with client private key
csr = x509.CertificateSigningRequestBuilder().subject_name(x509.Name([
    # Provide various details about who we are.
    x509.NameAttribute(NameOID.COUNTRY_NAME, u"US"),
    x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, u"California"),
    x509.NameAttribute(NameOID.LOCALITY_NAME, u"San Francisco"),
    x509.NameAttribute(NameOID.ORGANIZATION_NAME, u"My Company"),
    x509.NameAttribute(NameOID.COMMON_NAME, u"client1.com"),
])).add_extension(
    x509.SubjectAlternativeName([
        # Describe what sites we want this certificate for.
        x509.DNSName(u"mysite.com"),
        x509.DNSName(u"www.mysite.com"),
        x509.DNSName(u"subdomain.mysite.com"),
    ]),
    critical=False,
# Sign the CSR with our private key. is applicant's private key
).sign(rsa_key, hashes.SHA256())

# 3. send to certificate authority,  the data param name corresond fastapi parameter
response = requests.post(" http://127.0.0.1:8003/issue_cert",files={'csr_file':csr.public_bytes(serialization.Encoding.PEM)})
client1_cert = x509.load_pem_x509_certificate(response.content)
print("applicant: "+client1_cert.subject.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value)
print("issuer: "+client1_cert.issuer.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value)
print(client1_cert.not_valid_before)
print(client1_cert.not_valid_after)

applicant: client1.com
issuer: ca1_1
2021-11-26 01:11:25
2021-12-06 01:11:25


In [17]:
# cleint 2 cert sign by ca2_1
# 1. generate RSA key
rsa_key=util.generate_ras_key()

# 2. create certificate sign request and sign with client private key
csr = x509.CertificateSigningRequestBuilder().subject_name(x509.Name([
    # Provide various details about who we are.
    x509.NameAttribute(NameOID.COUNTRY_NAME, u"US"),
    x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, u"California"),
    x509.NameAttribute(NameOID.LOCALITY_NAME, u"San Francisco"),
    x509.NameAttribute(NameOID.ORGANIZATION_NAME, u"My Company"),
    x509.NameAttribute(NameOID.COMMON_NAME, u"client2.com"),
])).add_extension(
    x509.SubjectAlternativeName([
        # Describe what sites we want this certificate for.
        x509.DNSName(u"mysite.com"),
        x509.DNSName(u"www.mysite.com"),
        x509.DNSName(u"subdomain.mysite.com"),
    ]),
    critical=False,
# Sign the CSR with our private key. is applicant's private key
).sign(rsa_key, hashes.SHA256())

# 3. send to certificate authority,  the data param name corresond fastapi parameter
## notice here port is 8001
response = requests.post(f"http://127.0.0.1:{ca_ports['ca2_1']}/issue_cert",files={'csr_file':csr.public_bytes(serialization.Encoding.PEM)})
client2_cert = x509.load_pem_x509_certificate(response.content)
print("applicant: "+client2_cert.subject.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value)
print("issuer: "+client2_cert.issuer.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value)
print(client2_cert.not_valid_before)
print(client2_cert.not_valid_after)

applicant: client2.com
issuer: ca2_1
2021-11-26 01:11:25
2021-12-06 01:11:25


In [18]:
#verify chain
def verify_chain(cur_cert):
    while True:
        #eg, ca1_1
        subjective=cur_cert.subject.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value
        issuer_commonName=cur_cert.issuer.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value
        print(f"\nsubjective: {subjective} \t issuer: {issuer_commonName}")
        print(f"====check revoke status for {subjective} from {issuer_commonName}")
        print(f"http://127.0.0.1:{ca_ports[issuer_commonName]}/revoke_cert_status")
        response = requests.get(f"http://127.0.0.1:{ca_ports[issuer_commonName]}/revoke_cert_status",files={'crt_file':cur_cert.public_bytes(serialization.Encoding.PEM)})
        print(response.content)
        print(f"=====verfy cert using {issuer_commonName} public key")
        pub_key=ca_datas[issuer_commonName]['public_key']
        status=util.verify_cert_signature(cur_cert,pub_key)
        if status:
            print("correct signature")
        else:
            print("XXX wrong signaure XXX")
        if issuer_commonName=='ca_root':
            break
        
        cur_cert=ca_datas[issuer_commonName]['cert']
verify_chain(client1_cert)


subjective: client1.com 	 issuer: ca1_1
====check revoke status for client1.com from ca1_1
http://127.0.0.1:8003/revoke_cert_status
b'{"msg":"Not revoke"}'
=====verfy cert using ca1_1 public key
correct signature

subjective: ca1_1 	 issuer: ca1
====check revoke status for ca1_1 from ca1
http://127.0.0.1:8001/revoke_cert_status
b'{"msg":"Not revoke"}'
=====verfy cert using ca1 public key
correct signature

subjective: ca1 	 issuer: ca_root
====check revoke status for ca1 from ca_root
http://127.0.0.1:8000/revoke_cert_status
b'{"msg":"Not revoke"}'
=====verfy cert using ca_root public key
correct signature


In [19]:
# revoke client2 cert from ca1_1, will fail because not issue by it
cpb=client2_cert.public_bytes(serialization.Encoding.PEM)
response = requests.post(f"http://127.0.0.1:{ca_ports['ca1_1']}/revoke_cert",files={'crt_file':cpb})
print(response.content)


b'{"msg":"The certificate was not issue by this server, the issuer is ca2_1"}'


In [20]:
# revoke client1 cert from ca1_1
cpb=client1_cert.public_bytes(serialization.Encoding.PEM)
print(f"http://127.0.0.1:{ca_ports['ca1_1']}/revoke_cert")
response = requests.post(f"http://127.0.0.1:{ca_ports['ca1_1']}/revoke_cert",files={'crt_file':cpb})
print(response.content)

http://127.0.0.1:8003/revoke_cert
b'{"msg":"Revoke successful"}'


In [21]:
verify_chain(client1_cert)




subjective: client1.com 	 issuer: ca1_1
====check revoke status for client1.com from ca1_1
http://127.0.0.1:8003/revoke_cert_status
b'{"msg":"Already revoke"}'
=====verfy cert using ca1_1 public key
correct signature

subjective: ca1_1 	 issuer: ca1
====check revoke status for ca1_1 from ca1
http://127.0.0.1:8001/revoke_cert_status
b'{"msg":"Not revoke"}'
=====verfy cert using ca1 public key
correct signature

subjective: ca1 	 issuer: ca_root
====check revoke status for ca1 from ca_root
http://127.0.0.1:8000/revoke_cert_status
b'{"msg":"Not revoke"}'
=====verfy cert using ca_root public key
correct signature


In [22]:
verify_chain(client2_cert)


subjective: client2.com 	 issuer: ca2_1
====check revoke status for client2.com from ca2_1
http://127.0.0.1:8004/revoke_cert_status
b'{"msg":"Not revoke"}'
=====verfy cert using ca2_1 public key
correct signature

subjective: ca2_1 	 issuer: ca2
====check revoke status for ca2_1 from ca2
http://127.0.0.1:8002/revoke_cert_status
b'{"msg":"Not revoke"}'
=====verfy cert using ca2 public key
correct signature

subjective: ca2 	 issuer: ca_root
====check revoke status for ca2 from ca_root
http://127.0.0.1:8000/revoke_cert_status
b'{"msg":"Not revoke"}'
=====verfy cert using ca_root public key
correct signature


In [23]:
# get ca cert from ca2, the original cert
response = requests.get(f"http://127.0.0.1:{ca_ports['ca2']}/get_CA_cert")
print(response.content)

b'-----BEGIN CERTIFICATE-----\nMIIDNjCCAh6gAwIBAgIUStavnYFcy8yg2mz4/CKwJMH7XdkwDQYJKoZIhvcNAQEL\nBQAwWTELMAkGA1UEBhMCU0cxEjAQBgNVBAgMCVNpbmdhcG9yZTESMBAGA1UEBwwJ\nU2luZ2Fwb3JlMRAwDgYDVQQKDAdjYV9yb290MRAwDgYDVQQDDAdjYV9yb290MB4X\nDTIxMTEyNjAxMTA1M1oXDTIxMTIwNjAxMTA1M1owUTELMAkGA1UEBhMCU0cxEjAQ\nBgNVBAgMCVNpbmdhcG9yZTESMBAGA1UEBwwJU2luZ2Fwb3JlMQwwCgYDVQQKDANj\nYTIxDDAKBgNVBAMMA2NhMjCCASIwDQYJKoZIhvcNAQEBBQADggEPADCCAQoCggEB\nAN+QYcek2MlL/vgUN/QfK6h5SI958zRl0e3KcS3O981la+a0rVJ9d6QsQHAusj/x\nFV48w/sKVmLG5a4dE0oKCJTFB/wqdZiFCm3o9aabuPDJnnxlm8kxV5WI1GwwR9Uc\nBNvwaJ2C6n1DUTvKjhYQJWt+odmkeENtn/J0txsgUk0dGQeR7k1urgO9SxpT4m+D\nDv8LLQC+nxwZNOhSCWICy/OVg4tYybcTuTgtwOciagPHKG4cg/CsnWmnuallwtYn\n8yMdGhRJEHyWdDlcnwbiyspNRe3l4o8rrPZ2kaOe4vmHqBYOW/CeKmgCxClLylD7\nSWVK8vYxyFmCIDAtoDY+pGsCAwEAATANBgkqhkiG9w0BAQsFAAOCAQEAS1lBc5W6\nDb3dD/nz049YuPXehx9HIwJtS9GT6+ZxwKesEqiwDr1DaSSko2PwQGUbjH59Kzuu\nCZLfmrX61Mk2dEOLZEpLlLFK7QA7H9giVXvR0XPhEkPN/9VqeB7CzY6akNOKvtBx\nHw+HWZtcm5RZmn+P49INFvvajNGgxGs4UfKGmOQV03QdK

In [24]:
# revoke ca2 cert from ca_root
response=requests.post(f"http://127.0.0.1:{ca_ports['ca2']}/revoke_ca_cert")
print(response.content)

b'{"msg":"CA cert revoked, will get new cert on next request"}'


In [25]:
# verify again, get ca cert from ca2, the cert will different, this api will always return a cert, if delete, it will create a new one
response = requests.get(f"http://127.0.0.1:{ca_ports['ca2']}/get_CA_cert")
print(response.content)

b'-----BEGIN CERTIFICATE-----\nMIIDNjCCAh6gAwIBAgIUN/X7A7LU3imU++Reoi7vv/gPAUwwDQYJKoZIhvcNAQEL\nBQAwWTELMAkGA1UEBhMCU0cxEjAQBgNVBAgMCVNpbmdhcG9yZTESMBAGA1UEBwwJ\nU2luZ2Fwb3JlMRAwDgYDVQQKDAdjYV9yb290MRAwDgYDVQQDDAdjYV9yb290MB4X\nDTIxMTEyNjAxMTEyNloXDTIxMTIwNjAxMTEyNlowUTELMAkGA1UEBhMCU0cxEjAQ\nBgNVBAgMCVNpbmdhcG9yZTESMBAGA1UEBwwJU2luZ2Fwb3JlMQwwCgYDVQQKDANj\nYTIxDDAKBgNVBAMMA2NhMjCCASIwDQYJKoZIhvcNAQEBBQADggEPADCCAQoCggEB\nAN+QYcek2MlL/vgUN/QfK6h5SI958zRl0e3KcS3O981la+a0rVJ9d6QsQHAusj/x\nFV48w/sKVmLG5a4dE0oKCJTFB/wqdZiFCm3o9aabuPDJnnxlm8kxV5WI1GwwR9Uc\nBNvwaJ2C6n1DUTvKjhYQJWt+odmkeENtn/J0txsgUk0dGQeR7k1urgO9SxpT4m+D\nDv8LLQC+nxwZNOhSCWICy/OVg4tYybcTuTgtwOciagPHKG4cg/CsnWmnuallwtYn\n8yMdGhRJEHyWdDlcnwbiyspNRe3l4o8rrPZ2kaOe4vmHqBYOW/CeKmgCxClLylD7\nSWVK8vYxyFmCIDAtoDY+pGsCAwEAATANBgkqhkiG9w0BAQsFAAOCAQEAeWEzlpgA\nHQbd3ZGAN/UX0WhLWQNRUZjaNyODt2I8SL2HX4uu+TbDXtiCXlK8KtGpu27Jyfwq\ngpLSc/sTPa6kBG34DSLiOKjBU0W/YvG31b1Lyr7WKE/077qSCPZoeGX5Pp5pwIDR\njD7c84ItTSqZJufYp2Pz4uVxJKxPCAdG/+d0lpb1JKviY

In [26]:
verify_chain(client2_cert)


subjective: client2.com 	 issuer: ca2_1
====check revoke status for client2.com from ca2_1
http://127.0.0.1:8004/revoke_cert_status
b'{"msg":"Not revoke"}'
=====verfy cert using ca2_1 public key
correct signature

subjective: ca2_1 	 issuer: ca2
====check revoke status for ca2_1 from ca2
http://127.0.0.1:8002/revoke_cert_status
b'{"msg":"Not revoke"}'
=====verfy cert using ca2 public key
correct signature

subjective: ca2 	 issuer: ca_root
====check revoke status for ca2 from ca_root
http://127.0.0.1:8000/revoke_cert_status
b'{"msg":"Already revoke"}'
=====verfy cert using ca_root public key
correct signature


In [29]:
# openssl req -newkey rsa:2048 -keyout PRIVATEKEY.key -out MYCSR.csr

f=open('MYCSR.csr','rb')
openssl_csr=x509.load_pem_x509_csr(f.read())
print(openssl_csr.subject.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value)
response = requests.post(f"http://127.0.0.1:{ca_ports['ca1_1']}/issue_cert",files={'csr_file':openssl_csr.public_bytes(serialization.Encoding.PEM)})
client2_cert = x509.load_pem_x509_certificate(response.content)
print("applicant: "+client2_cert.subject.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value)
print("issuer: "+client2_cert.issuer.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value)
print(client2_cert.not_valid_before)
print(client2_cert.not_valid_after)

openssl
applicant: openssl
issuer: ca1_1
2021-11-26 02:01:35
2021-12-06 02:01:35
